In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go
from ipywidgets import widgets
import seaborn as sns
import folium
from folium.plugins import TimestampedGeoJson
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


In [ ]:
#Reading Needed files
df=pd.read_csv("/kaggle/input/air-pollution-dataset-india20162018/india_noNA.csv")

In [ ]:
#data for diwali of the three years
#2016 - Sun 30 Oct
#2017 - Thu 19 Oct
#2018 - Wed 7 Nov

#2016 dates - 28,29,30,31,1 Oct-Nov
#2017 dates - 17,18,19,20,21 Oct
#2018 dates - 5,6,7,8,9 Nov

#2016 -> 20,21,22,23,24,25,26,27,28,29, 30 ,31,1,2,3,4,5,6,7,8,9
#2017 -> 9,10,11,12,13,14,15,16,17,18, 19 ,20,21,22,23,24,25,26,27,28,29

#pollutants released by -
#bursting of crackers - ozone, no2, so2
#burning of fuels - pm, co, no2, so2

In [ ]:
#grouping the data to get one value per city per pollutant per day
data_daily_diwali=df
data_daily_diwali=data_daily_diwali.drop(['utc','country','attribution','location','unit'],axis=1)
data_daily_diwali['local'] = data_daily_diwali['local'].map(lambda x: str(x)[:10])
data_daily_diwali['local']=pd.to_datetime(data_daily_diwali['local'],format="%Y-%m-%d")

agg_daily_diwali=data_daily_diwali
grouped_daily_diwali=agg_daily_diwali.groupby(['local','city','parameter'])
grouped_daily_diwali=grouped_daily_diwali.mean()

grouped_daily_diwali.insert(3,'date',pd.to_datetime('2016-01'))
grouped_daily_diwali.insert(4,'city',0)
grouped_daily_diwali.insert(5,'parameter',0)

for i in grouped_daily_diwali.index:
    grouped_daily_diwali.date[i]=i[0]
    grouped_daily_diwali.city[i]=i[1]
    grouped_daily_diwali.parameter[i]=i[2]
grouped_daily_diwali

In [ ]:
#getting dates for diwali days
y16_oct = list(range(20,32))
y16_nov = list(range(1,10))
y17 = list(range(9,30))

y_comb = [y16_oct, y16_nov, y17]
year_month = ["2016-10-", "2016-11-", "2017-10-"]
dat = []
for i in range(len(y_comb)):
    for j in range(len(y_comb[i])):
        if (y_comb[i][j] < 10):
            y_comb[i][j] = '0' + str(y_comb[i][j])
        else:
            y_comb[i][j] = str(y_comb[i][j])

print("Y combined = ", y_comb)

In [ ]:
#Dropdown for diwali, based on city and pollutant
#The value at x = 10 (middle of the graph) is the pollutant level on diwali
#We have considered values 10 days before and after to see the trend for that period

from ipywidgets import widgets
#-----------------------------------------creating temp dataframe--------------------------

#initially choosing the city as Delhi, and pollutant as NO2
df_sub = grouped_daily_diwali.loc[grouped_daily_diwali['city'] == 'Delhi']
df_sub = df_sub.loc[df_sub['parameter'] == 'no2']

#Iterating through the entire list of dates (list created earlier)
#Getting the data for that day and the corresponding chosen parameters
years_data = []
for ym in range(len(year_month)):
    y = []
    for d in y_comb[ym]:
        c = year_month[ym] + d
        df1_sub = df_sub[df_sub['date'].astype(str).str.contains(c)]
        yd = df1_sub.value.mean()
        y.append(yd)
    years_data.append(y)

#Merging the data for 2016 that had been split over two months
y16 = []
for i in years_data[0]:
    y16.append(i)
for i in years_data[1]:
    y16.append(i)

y17_data = []
for i in years_data[2]:
    y17_data.append(i)
#------------------------------------------------------------------------------------------

city_diwali = widgets.Dropdown(
    description='City:   ',
    value='Delhi',
    options=grouped_daily_diwali['city'].unique().tolist()
)
parameter_diwali = widgets.Dropdown(
    options=list(grouped_daily_diwali['parameter'].unique()),
    value='no2',
    description='Parameter:   ',
)

#Adding different traces for each year 
#Also highlighting the subtraces for the five days of diwali for that particular year

trace1_diwali = go.Scatter(y=y16, mode='lines', name='year 2016', line=dict(color='rgb(255, 133, 133)'))
trace2_diwali = go.Scatter(y=y17_data, mode='lines', name='Year 2017', line=dict(color='rgb(16, 216, 235)'))
trace3_diwali = go.Scatter(y=y16[8:13], x = list(range(8,13)), name = "Diwali days 2016", line=dict(color='rgb(250, 5, 5)'))
trace4_diwali = go.Scatter(y=y17_data[8:13], x = list(range(8,13)), name = "Diwali days 2017", line=dict(color='rgb(12, 10, 128)'))

g_diwali = go.FigureWidget(data=[trace1_diwali, trace2_diwali, trace3_diwali, trace4_diwali],
                    layout=go.Layout(
                        title=dict(
                            text='Pollutant levels during diwali days for 2016,2017'
                        )
                    ))

#Function to run when the chosen parameter changes in the dropdown
def response(change):
        #-----------------------------------------creating temp dataframe--------------------------
        df_sub = grouped_daily_diwali.loc[grouped_daily_diwali['city'] == city_diwali.value]
        df_sub = df_sub.loc[df_sub['parameter'] == parameter_diwali.value]

        years_data = []
        for ym in range(len(year_month)):
            y = []
            for d in y_comb[ym]:
                c = year_month[ym] + d
                df1_sub = df_sub[df_sub['date'].astype(str).str.contains(c)]
                if (len(df1_sub) == 0):
                    print("No data")
                    #yd = np.nan
                else: 
                    yd = df1_sub.value.mean()
                    y.append(yd)
            years_data.append(y)

        y16 = []
        for i in years_data[0]:
            y16.append(i)
        for i in years_data[1]:
            y16.append(i)

        y17_data = []
        for i in years_data[2]:
            y17_data.append(i)        
        #------------------------------------------------------------------------------------------
        y0 = y16
        y1 = y17_data
        y2 = y16[8:13]
        x2 = list(range(8,13))
        y3 = y17_data[8:13]
        x3 = list(range(8,13))
        with g_diwali.batch_update():
            g_diwali.data[0].y = y0
            g_diwali.data[1].y = y1
            g_diwali.data[2].y = y2
            g_diwali.data[3].y = y3
            g_diwali.data[2].x = x2
            g_diwali.data[3].x = x3
            g_diwali.layout.xaxis.title = 'Date'
            g_diwali.layout.yaxis.title = 'Pollutant levels'


city_diwali.observe(response, names="value")
parameter_diwali.observe(response, names="value")
container2_diwali = widgets.HBox([city_diwali, parameter_diwali])
widgets.VBox([container2_diwali,g_diwali])

In [ ]:
#function to get the day from the given date
from datetime import date
import calendar

def get_day(my_date):
    my_date = str(my_date)
    y = int(my_date[:4])
    m = int(my_date[5:7])
    d = int(my_date[8:10])

    my_day = date(y, m, d).weekday()
    return calendar.day_name[my_day]

In [ ]:
#Comparison of pollutant levels on weekends and weekdays over time

from ipywidgets import widgets
import plotly.graph_objects as go

#-----------------------------------------creating temp dataframe--------------------------

#Initially choosing the city as Delhi, pollutant as NO2, year as 2016
weekday = pd.DataFrame()
weekend = pd.DataFrame()
df_no2 = grouped_daily_diwali.loc[grouped_daily_diwali['parameter'] == 'no2']
df_no2 = df_no2.loc[df_no2['city'] == 'Delhi']
df_no2 = df_no2[df_no2['date'].astype(str).str.contains("2016")]

#Iterating through the data frome and checking if the day was weekday/weekend
#Splitting the data into two dataframes for weekends and weekdays

for i in range(len(df_no2)):
    if ((get_day(df_no2.iloc[i,3]) == 'Sunday') or (get_day(df_no2.iloc[i,3]) == 'Saturday')):
        weekend = weekend.append(df_no2.iloc[i,:])
    else:
        weekday = weekday.append(df_no2.iloc[i,:])
#------------------------------------------------------------------------------------------

city_ww = widgets.Dropdown(
    description='City:   ',
    value='Delhi',
    options=grouped_daily_diwali['city'].unique().tolist()
)

parameter_ww = widgets.Dropdown(
    options=list(grouped_daily_diwali['parameter'].unique()),
    value='no2',
    description='Parameter:   ',
)

year_ww = widgets.Dropdown(
    options=["2016","2017","2018"],
    value='2016',
    description='Year:   ',
)

trace1_ww = go.Bar(x=weekend['date'], y=weekend['value'], name='Weekends')
trace2_ww = go.Bar(x=weekday['date'], y=weekday['value'], name='Weekdays')
g_ww = go.FigureWidget(data=[trace1_ww, trace2_ww],
                    layout=go.Layout(
                        title=dict(
                            text='Pollutant levels on weekends vs weekdays'
                        )
                    ))
def response(change):

        #-----------------------------------------creating temp dataframe--------------------------
        weekday = pd.DataFrame()
        weekend = pd.DataFrame()
        df_no2 = grouped_daily_diwali.loc[grouped_daily_diwali['parameter'] == parameter_ww.value]
        df_no2 = df_no2.loc[df_no2['city'] == city_ww.value]
        df_no2 = df_no2[df_no2['date'].astype(str).str.contains(year_ww.value)]

        for i in range(len(df_no2)):
            if ((get_day(df_no2.iloc[i,3]) == 'Sunday') or (get_day(df_no2.iloc[i,3]) == 'Saturday')):
                weekend = weekend.append(df_no2.iloc[i,:])
            else:
                weekday = weekday.append(df_no2.iloc[i,:])
        #------------------------------------------------------------------------------------------

        #print("Printing data = ")
        #print("Weekend = ", weekend)
        #print("Weekday = ", weekday)
        if (len(weekday) == 0 or len(weekend) == 0):
            print("There's no data for this combination of parameters")
        else :
            x0 = weekend['date']
            y0 = weekend['value']
            x1 = weekday['date']
            y1 = weekday['value']
            with g_ww.batch_update():
                g_ww.data[0].x = x0
                g_ww.data[0].y = y0
                g_ww.data[1].x = x1
                g_ww.data[1].y = y1
                g_ww.layout.xaxis.title = 'Date'
                g_ww.layout.yaxis.title = 'Pollutant levels of ' + parameter_ww.value

city_ww.observe(response, names="value")
parameter_ww.observe(response, names="value")
year_ww.observe(response, names="value")
container2_ww = widgets.HBox([city_ww, parameter_ww,year_ww])
widgets.VBox([container2_ww,g_ww])

In [ ]:
#Box plot for pollutant levels on weekends and weekdays

from ipywidgets import widgets
import plotly.graph_objects as go

#-----------------------------------------creating temp dataframe--------------------------
weekday_box = pd.DataFrame()
weekend_box = pd.DataFrame()
df_no2_box = grouped_daily_diwali.loc[grouped_daily_diwali['parameter'] == 'no2']
df_no2_box = df_no2_box.loc[df_no2_box['city'] == 'Delhi']
df_no2_box = df_no2_box[df_no2_box['date'].astype(str).str.contains("2016")]

for i in range(len(df_no2_box)):
    if ((get_day(df_no2_box.iloc[i,3]) == 'Sunday') or (get_day(df_no2_box.iloc[i,3]) == 'Saturday')):
        weekend_box = weekend_box.append(df_no2_box.iloc[i,:])
    else:
        weekday_box = weekday_box.append(df_no2_box.iloc[i,:])
#------------------------------------------------------------------------------------------

city_ww_box = widgets.Dropdown(
    description='City:   ',
    value='Delhi',
    options=grouped_daily_diwali['city'].unique().tolist()
)

parameter_ww_box = widgets.Dropdown(
    options=list(grouped_daily_diwali['parameter'].unique()),
    value='no2',
    description='Parameter:   ',
)

year_ww_box = widgets.Dropdown(
    options=["2016","2017","2018"],
    value='2016',
    description='Year:   ',
)

trace0_ww_box = go.Box(y=weekday_box['value'], name = 'Weekdays')
trace1_ww_box = go.Box(y=weekend_box['value'], name = 'Weekends')
g_ww_box = go.FigureWidget(data=[trace0_ww_box, trace1_ww_box],
                    layout=go.Layout(
                        title=dict(
                            text='Pollutant levels on weekends vs weekdays'
                        )
                    ))
def response(change):

        #-----------------------------------------creating temp dataframe--------------------------
        weekday_box = pd.DataFrame()
        weekend_box = pd.DataFrame()
        df_no2_box = grouped_daily_diwali.loc[grouped_daily_diwali['parameter'] == parameter_ww_box.value]
        df_no2_box = df_no2_box.loc[df_no2_box['city'] == city_ww_box.value]
        df_no2_box = df_no2_box[df_no2_box['date'].astype(str).str.contains(year_ww_box.value)]

        for i in range(len(df_no2_box)):
            if ((get_day(df_no2_box.iloc[i,3]) == 'Sunday') or (get_day(df_no2_box.iloc[i,3]) == 'Saturday')):
                weekend_box = weekend_box.append(df_no2_box.iloc[i,:])
            else:
                weekday_box = weekday_box.append(df_no2_box.iloc[i,:])
        #------------------------------------------------------------------------------------------
        #print("printing now ..............")
        #print(weekday_box)
        if (len(weekday_box) == 0 or len(weekend_box) == 0):
            print("Got no rows")
        else :
            y0 = weekday_box['value']
            y1 = weekend_box['value']
            with g_ww_box.batch_update():
                g_ww_box.data[0].y = y0
                g_ww_box.data[1].y = y1
                g_ww_box.layout.yaxis.title = 'Pollutant levels of ' + parameter_ww_box.value


city_ww_box.observe(response, names="value")
parameter_ww_box.observe(response, names="value")
year_ww_box.observe(response, names="value")
container2_ww_box = widgets.HBox([city_ww_box, parameter_ww_box,year_ww_box])
widgets.VBox([container2_ww_box,g_ww_box])

In [ ]:
#Checking the variation of different pollutant levels for a particular day (here - 7th March 2016)
#Done for a particular city - Delhi, location - RK Puram

from ipywidgets import widgets

#-----------------------------------------creating temp dataframe--------------------------
c = "2016-03-07"
df_sub = df[df['local'].str.contains(c)]
df_sub = df_sub.loc[df_sub['city'] == 'Delhi']
df_sub = df_sub.loc[df_sub['location'] == 'RK Puram']
df_temp = df_sub.loc[df_sub['parameter'] == 'no2']
#------------------------------------------------------------------------------------------

parameter_d = widgets.Dropdown(
    options=list(df['parameter'].unique()),
    value='no2',
    description='Parameter:   ',
)

trace1_d = go.Scatter(x=df_temp['local'], y=df_temp['value'], mode='lines')
g_d = go.FigureWidget(data=[trace1_d],
                    layout=go.Layout(
                        title=dict(
                            text='Pollutant levels'
                        )
                    ))
def response(change):

        #-----------------------------------------creating temp dataframe--------------------------
        df_temp = df_sub.loc[df_sub['parameter'] == parameter_d.value]
        #------------------------------------------------------------------------------------------

        x0 = df_temp['local']
        y0 = df_temp['value']
        with g_d.batch_update():
            g_d.data[0].x = x0
            g_d.data[0].y = y0
            g_d.layout.xaxis.title = 'Date'
            g_d.layout.yaxis.title = 'Pollutant levels of ' + parameter_d.value


parameter_d.observe(response, names="value")
container2_d = widgets.HBox([parameter_d])
widgets.VBox([container2_d,g_d])

In [ ]:
'''

Inferences -

1) We had wanted to see if there was any difference between the pollutant levels on weekdays as compared to that on the
weekends. The plot over time for different years show that there is not much difference between weekdays and weekends.
The box plot for the different cities also shows the same. The pollutant levels seem to be slightly higher for cities
such as Delhi for the pollutant NO2 in the year 2016. 

But overall they both seem to follow the same trend as shown by the bar chart. Another noticeable change is that the 
SO2 concentration in 2016, Delhi is more spread out on weekdays but is more concentrated over a small range on the 
weekends and the median is slightly lower.

2) If we check the pm2.5 values for the year 2016, Delhi, we notice that the value is extremely high (around 700)
on 6th Nov. If we look up what happened on this day we notice that on 6th Nov the Indian government declared levels of 
air pollution in Delhi an emergency situation, closing schools and construction sites. The level of PM2.5 pollutants, 
which are the most harmful because they can reach deep into the lungs and breach the blood-brain barrier, had reached 
values more than 16 times the safe limit of 60.

3) The CO concentration on 3rd Dec 2017 was tremendously high compared to the neighbouring values. Also we found that on 
this day there was a cricket match organised between India and Sri Lanka. And due to the air quality in Delhi on that day, 
Sri Lankan players were forced to wear masks and many complained of breathing trouble. This validates our observations.

4) Looking at the pm2.5 and NO2 values for 2017, we can observe the that the values start increasing on 8 Nov, reach very
high levels on 9, 10 and 11 Nov (around 140). Also we noticed that Nov 8 w

5) Diwali is when we expect the pollution levels to spike. So we wanted to see if there was any difference in pollutant 
concentrations over the years during the five days of diwali. A plot of the concentrations of various pollutants was 
considered. This was done for the year 2016 and 2017. Bursting of crackers mainly releases NO2, O3, SO2. Observing these 
values from the graph clearly shows that the pollution had dropped in 2017 as compared to 2016 for Delhi. Even other cities
like Mumbai, Kolkata have smaller values of NO2 in 2017 than 2016. Thus we can say that people have become more aware of the
deteriorating air quality and are likely to be taking steps to help improve the situation.

6) Looking at the variation during a day, we can observe certain pollutants like NO2 (released due to incomplete combustion 
of fuels) showed an increase in concentration at around 8:30 am and again at 5:00 pm. This shows that it increases at 
approximately the commute time of most people. Similarly, SO2 increases at around 5:00pm 

'''